In [ ]:

# get all the recordings on that day
mouse = 'M24029'
date = '20240426'
import os
import subprocess
subprocess.run('ulimit -n 4096',shell=True)
#grab recordings from the server to local machine (Beast)
base_folder = '/mnt/rds01/ibn-vision/DATA/SUBJECTS/'
job_kwargs = dict(n_jobs=32, chunk_duration='1s', progress_bar=True)

ephys_folder = base_folder + mouse + '/ephys/' + date +'/'
# allocate destination folder and move the ephys folder on the server to Beast lab user
from pathlib import Path
import platform
import os
import shutil
import argparse
import numpy as np
import matplotlib.pyplot as plt
import spikeinterface.sorters
import spikeinterface.full as si
import  scipy.signal
import spikeinterface.extractors as se
import spikeinterface.comparison
import spikeinterface.exporters
import spikeinterface.curation
import spikeinterface.widgets 



from datetime import datetime
startTime = datetime.now()
print('Start Time:' + startTime.strftime("%m/%d/%Y, %H:%M:%S"))
''' this section defines the animal and dates and fetch the recordings from the server to Beast'''
print('copying ephys data from:' + ephys_folder)

#load first probe from beast folder - MEC probe for Diao
probe0_raw = si.read_spikeglx(ephys_folder,stream_name='imec0.ap')
print(probe0_raw)

probe0_highpass = si.highpass_filter(probe0_raw,freq_min=300.)
bad_channel_ids, channel_labels = si.detect_bad_channels(probe0_highpass)
probe0_remove_channels = probe0_highpass.remove_channels(bad_channel_ids)
print('probe0_bad_channel_ids',bad_channel_ids)
probe0_phase_shift = si.phase_shift(probe0_remove_channels)
probe0_common_reference = si.common_reference(probe0_phase_shift,operator='median',reference='global')
probe0_preprocessed = probe0_common_reference
probe0_cat = si.concatenate_recordings([probe0_preprocessed])
print('probe0_preprocessed',probe0_preprocessed)
print('probe0 concatenated',probe0_cat)

recording_corrected, motion, motion_info = si.correct_motion(
        recording=probe0_cat, preset="nonrigid_accurate", folder=ephys_folder + 'probe0_motion', output_motion=True, output_motion_info=True, **job_kwargs
    )

# load
folder = ephys_folder + 'probe0_motion'
motion_info = si.load_motion_info(folder)

# and plot
fig = plt.figure(figsize=(14, 8))
si.plot_motion_info(
    motion_info, probe0_cat,
    figure=fig,
    depth_lim=(400, 600),
    color_amplitude=True,
    amplitude_cmap="inferno",
    scatter_decimate=10,
)
#load second probe from beast folder - LEC probe for Diao
probe1_raw = si.read_spikeglx(ephys_folder,stream_name='imec1.ap')
print(probe1_raw)

probe1_highpass = si.highpass_filter(probe1_raw,freq_min=300.)
bad_channel_ids, channel_labels = si.detect_bad_channels(probe1_highpass)
probe1_remove_channels = probe1_highpass.remove_channels(bad_channel_ids)
print('probe1_bad_channel_ids',bad_channel_ids)
probe1_phase_shift = si.phase_shift(probe1_remove_channels)
probe1_common_reference = si.common_reference(probe1_phase_shift,operator='median',reference='global')
probe1_preprocessed = probe1_common_reference
probe1_cat = si.concatenate_recordings([probe1_preprocessed])
print('probe1_preprocessed',probe1_preprocessed)
print('probe1 concatenated',probe1_cat)

recording_corrected, motion, motion_info = si.correct_motion(
    recording=probe1_cat, preset="nonrigid_accurate", folder=ephys_folder + 'probe1_motion', output_motion=True, output_motion_info=True, **job_kwargs
    )

# load
folder = ephys_folder + 'probe1_motion'
motion_info = si.load_motion_info(folder)

# and plot
fig = plt.figure(figsize=(14, 8))
si.plot_motion_info(
    motion_info, probe1_cat,
    figure=fig,
    depth_lim=(400, 600),
    color_amplitude=True,
    amplitude_cmap="inferno",
    scatter_decimate=10,
)